In [1]:
import cPickle
import os.path

api_key = cPickle.load( file( os.path.expanduser( '~/mediacloud_api_key.pickle' ), 'r' ) )

In [2]:
import cPickle
import os.path

cPickle.dump( api_key, file( os.path.expanduser( '~/mediacloud_api_key.pickle' ), 'wb' ) )

In [7]:
import mediacloud, json

mc = mediacloud.api.AdminMediaCloud( api_key)


In [33]:
for k in sorted(mc.storyCoreNlpList(story_id)[0]['corenlp'].keys()):
    for w in mc.storyCoreNlpList(story_id)[0]['corenlp'][k]['corenlp']['sentences'][0]['tokens']:
        print w['word'],
    print
       

Zoe Quinn 's Depression Quest
Credit Illustration by Istvan Banyai
Earlier this month , an anonymous message was posted to the discussion-board Web site 4chan .
In it , the author threatened to hurt the video-game developer Zoe Quinn : `` Next time she shows up at a conference we ... give her a crippling injury that 's never going to fully heal ... a good solid injury to the knees .
I 'd say a brain damage , but we do n't want to make it so she ends up too retarded to fear us . ''
For the past eighteen months , the twenty-seven-year-old Quinn has received a cavalcade of similar threats , which have created an ambient hum of menace in her life , albeit one that she has mostly been able to ignore .
But at the end of August she was `` doxed , '' a slang term for document tracing , which is when a person 's personal details -- home address , phone numbers , bank details , and , in some cases , social-security number -- are made public on the Internet .
Doxing carries with it a tacit invita

KeyboardInterrupt: 

In [5]:
def corenlp_annotated( s ):
    corenlp = s['corenlp']
    
    if 'annotated' not in corenlp:
        return True
    
    if corenlp['annotated'] == 'false':
        return False
    
    return True

In [6]:
#import cliff
from mediameter.cliff import Cliff

cliff_api = Cliff('http://civicdev.media.mit.edu', 8080)

ImportError: No module named mediameter.cliff

# Flags

In [6]:
reload_stories = False
reprocess_with_cliff = False


In [7]:
import shelve

In [8]:
controversy_stories = []

fetch_size = 100

if reload_stories:
    
    last_processed_stories_id = 0

    controversy_stories_shelve = shelve.open((os.path.expanduser( '~/Dropbox/gamergate_data/controversy_stories_shelve.shelve' ) ))
    
    while True:
        print "fetching with last_processed_stories_id", last_processed_stories_id
        stories = mc.storyList( solr_query='{~ controversy:1113 }', rows=fetch_size, corenlp=True, 
                               last_processed_stories_id=last_processed_stories_id, sentences=True, text=True)

        if len( stories ) == 0:
            break
            
        for s in stories:          
            
            #del s['description']
            
            controversy_stories_shelve[ str(s[ 'stories_id']) ] = s
            
        last_processed_stories_id = stories[-1]['processed_stories_id']
        
        #if len( controversy_stories_shelve.keys()) > 200:
        #    break

    print len ( controversy_stories_shelve.keys())
    controversy_stories_shelve.close()


In [9]:
if reprocess_with_cliff:
    
    controversy_stories_shelve  = shelve.open(os.path.expanduser( '~/Dropbox/gamergate_data/controversy_stories_shelve.shelve' ), flag='r' )
    controversy_stories_corenlp = shelve.open(os.path.expanduser( '~/Dropbox/gamergate_data/controversy_stories_corenlp.shelve' ) , flag='n' )

    stories_ids = sorted( controversy_stories_shelve.keys() )

    non_corenlp_annotated_stories = []
    cliff_error_stories = []
    cliff_no_result_stories = []
    cliff_no_people_stories = []

    for stories_id in stories_ids:
        s = controversy_stories_shelve[ stories_id]

        cliff_info = {}

        if corenlp_annotated( s ):
            try:
                cliff_info['cliff_results'] = cliff_api.parseNlpJson( s['corenlp'])
            except:
                print 'Cliff Error on ', stories_id

                cliff_error_stories.append( stories_id )

                continue

            if 'results' not in cliff_info['cliff_results']:
                cliff_no_result_stories.append( stories_id )

            else:

                results = cliff_info['cliff_results']['results']

                if 'people' in results:
                    people = results['people']
                    names = [ p['name'] for p in people ]

                    cliff_info[ 'names'] = names

                else:
                    cliff_no_people_stories.append( stories_id )

            controversy_stories_corenlp[ stories_id ] = cliff_info

        else:  # not corenlp_annotated

            print stories_id, 'not annotated'

            non_corenlp_annotated_stories.append( stories_id )

    cliff_problems = shelve.open(os.path.expanduser( '~/Dropbox/gamergate_data/cliff_problmes.shelve' ) , flag='n' )

    cliff_problems[ 'cliff_error_stories'] = cliff_error_stories
    cliff_problems[ 'cliff_no_result_stories'] = cliff_no_result_stories
    cliff_problems[ 'cliff_no_people_stories'] = cliff_no_people_stories
    cliff_problems[ 'non_corenlp_annotated_stories'] = non_corenlp_annotated_stories
    cliff_problems.close()
    controversy_stories_corenlp.close()
    controversy_stories_shelve.close()

In [10]:
controversy_stories_shelve  = shelve.open(os.path.expanduser( '~/Dropbox/gamergate_data/controversy_stories_shelve.shelve' ), flag='r' )
controversy_stories_corenlp = shelve.open(os.path.expanduser( '~/Dropbox/gamergate_data/controversy_stories_corenlp.shelve' ) , flag='r' )
cliff_problems = shelve.open(os.path.expanduser( '~/Dropbox/gamergate_data/cliff_problmes.shelve' ) , flag='r' )

print len ( controversy_stories_shelve.keys()), 'total stories'
print len (controversy_stories_corenlp.keys() ), 'controvery_stories corenlp annotated'
print "\t", len ( cliff_problems['cliff_error_stories']), 'cliff error stories'
print "\t", len ( cliff_problems['cliff_no_result_stories']), 'cliff no result stories'
print "\t", len ( cliff_problems['cliff_no_people_stories']), 'cliff no people stories'
print len ( cliff_problems['non_corenlp_annotated_stories'] ), 'non corenlp annotated stories'



16048 total stories
14850 controvery_stories corenlp annotated
	376 cliff error stories
	1 cliff no result stories
	0 cliff no people stories
822 non corenlp annotated stories


In [2]:
for i in controversy_stories_corenlp.keys():
    print i

NameError: name 'controversy_stories_corenlp' is not defined

In [12]:
import itertools
from collections import Counter

edge_counts = Counter()
name_counts = Counter()
stories_processed = 0

for stories_id in controversy_stories_corenlp.keys():
    if 'names' in controversy_stories_corenlp[ stories_id ]:
        for name in controversy_stories_corenlp[ stories_id ] [ 'names' ]:
            if name not in name_counts:
                name_counts[ name ] = 0

            name_counts[ name ] += 1
        
        #edges = sorted( list (itertools.combinations( controversy_stories_corenlp[ stories_id ] [ 'names' ], 2 ) ) )

        #print edges

        #for edge in edges:
        #    if edge not in edges:
        #        edge_counts[edge] = 0

        #    edge_counts[ edge ] += 1
                
    stories_processed += 1
    
    if stories_processed % 100 == 0:
        print stories_processed, "stories processed"
        print len( name_counts.keys() ), "names"
        print name_counts.most_common(5)
        
        #break

100 stories processed
752 names
[(u'Zoe Quinn', 11), (u'Anita Sarkeesian', 9), (u'Leigh Alexander', 8), (u'Adam Baldwin', 5), (u'Eron Gjoni', 4)]
200 stories processed
1155 names
[(u'Zoe Quinn', 23), (u'Anita Sarkeesian', 23), (u'Leigh Alexander', 10), (u'Brianna Wu', 9), (u'Adam Baldwin', 8)]
300 stories processed
1642 names
[(u'Anita Sarkeesian', 32), (u'Zoe Quinn', 30), (u'Brianna Wu', 15), (u'Leigh Alexander', 13), (u'Adam Baldwin', 9)]
400 stories processed
2128 names
[(u'Anita Sarkeesian', 44), (u'Zoe Quinn', 43), (u'Brianna Wu', 21), (u'Leigh Alexander', 15), (u'Adam Baldwin', 14)]
500 stories processed
2897 names
[(u'Anita Sarkeesian', 57), (u'Zoe Quinn', 54), (u'Brianna Wu', 27), (u'Leigh Alexander', 21), (u'Adam Baldwin', 14)]
600 stories processed
3299 names
[(u'Anita Sarkeesian', 69), (u'Zoe Quinn', 65), (u'Brianna Wu', 32), (u'Leigh Alexander', 23), (u'Adam Baldwin', 16)]
700 stories processed
3711 names
[(u'Anita Sarkeesian', 87), (u'Zoe Quinn', 77), (u'Brianna Wu', 41), 

In [13]:
top_400_names = set([ n[0] for n in name_counts.most_common(400) ])

edge_counts = Counter()
stories_processed = 0

for stories_id in controversy_stories_corenlp.keys():
    if 'names' in controversy_stories_corenlp[ stories_id ]:

        story_top_names = [ name for name in controversy_stories_corenlp[ stories_id ] [ 'names' ]  if name in top_400_names]
        
        if len( story_top_names ) > 1 :
            #print story_top_names
            
            edges = sorted( list (itertools.combinations( story_top_names, 2 ) ) )

            edges = [ tuple( sorted(e) ) for e in edges ]

            for edge in edges:
                if edge not in edges:
                    edge_counts[edge] = 0

                edge_counts[ edge ] += 1
                
    stories_processed += 1
    
    if stories_processed % 1000 == 0:
        print stories_processed, "stories processed"
        print len( edge_counts.keys() ), "names"
        print edge_counts.most_common(5)
        
        #break


1000 stories processed
5789 names
[((u'Anita Sarkeesian', u'Zoe Quinn'), 64), ((u'Anita Sarkeesian', u'Brianna Wu'), 34), ((u'Brianna Wu', u'Zoe Quinn'), 32), ((u'Anita Sarkeesian', u'Leigh Alexander'), 20), ((u'Leigh Alexander', u'Zoe Quinn'), 17)]
2000 stories processed
7143 names
[((u'Anita Sarkeesian', u'Zoe Quinn'), 117), ((u'Anita Sarkeesian', u'Brianna Wu'), 61), ((u'Brianna Wu', u'Zoe Quinn'), 57), ((u'Anita Sarkeesian', u'Leigh Alexander'), 41), ((u'Eron Gjoni', u'Zoe Quinn'), 34)]
3000 stories processed
8299 names
[((u'Anita Sarkeesian', u'Zoe Quinn'), 174), ((u'Anita Sarkeesian', u'Brianna Wu'), 103), ((u'Brianna Wu', u'Zoe Quinn'), 94), ((u'Anita Sarkeesian', u'Leigh Alexander'), 55), ((u'Leigh Alexander', u'Zoe Quinn'), 49)]
4000 stories processed
9468 names
[((u'Anita Sarkeesian', u'Zoe Quinn'), 226), ((u'Anita Sarkeesian', u'Brianna Wu'), 132), ((u'Brianna Wu', u'Zoe Quinn'), 120), ((u'Anita Sarkeesian', u'Leigh Alexander'), 69), ((u'Adam Baldwin', u'Zoe Quinn'), 63)]
50

In [14]:
len( top_400_names )
len( edge_counts.keys() )
sum(edge_counts.values())

for edge in edge_counts.keys():
    assert edge[0] < edge[1]
    
assert Counter( [ eg[0] + "," + eg[1] for eg in sorted( edge_counts.keys() ) ] ).most_common( 1 )[0][1] == 1


In [15]:
import networkx as nx

person_graph = nx.Graph()

for edge in sorted(edge_counts.keys()):
    person_graph.add_edge( edge[0], edge[1], weight=edge_counts[edge])
    

len( list( nx.connected_components( person_graph) )[0] )
#person_graph.edges()
    
    

400

In [16]:
nx.write_gexf( person_graph, os.path.expanduser( '~/Dropbox/gamergate_data/person_graph.gexf' ) )

In [17]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy
